In [ ]:
import os
if os.path.isdir('../../UnifiedOntology-Pipeline'): os.chdir('../')

In [ ]:
import import_ipynb
from tool.extractClass import *
from pipeline.connectIndividuals import getNamespace

In [ ]:
loginHuggingFace()

In [ ]:
path_to_onto = "./pipeline/3_mergeOutput/UnifiedOntology.ttl"
df = getOntoClasses(path_to_onto)

In [ ]:
name_tokenizer = "bert-base-uncased"
name_model = "OhWayTee/bert-taxonomy"
name_pipeline = "text-classification"

In [ ]:
classifier = getClassifier(name_tokenizer, name_model, name_pipeline)

In [ ]:
df = df[:10]

In [ ]:
df_pred = getPredictions(classifier, df)

In [ ]:
df_top = sortPredictions(df_pred)

In [ ]:
df_top

In [ ]:
df_superclass = df_top[df_top['label'] == "LABEL_1"]
df_subclass = df_top[df_top['label'] == "LABEL_2"]

In [ ]:
g = Graph()
g.parse(path_to_onto, format='turtle')

In [ ]:
getNamespace(g)

In [ ]:
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


"""